<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# AutoGen Orchestrator Agent

In this tutorial, we'll explore orchestrator agent workflows with [AutoGen GroupChats](https://microsoft.github.io/autogen/dev//user-guide/core-user-guide/design-patterns/group-chat.html).

**Agent orchestration** enables collaboration among multiple specialized agents, activating only the most relevant one based on the current subtask context. Instead of relying on a fixed sequence, agents dynamically participate depending on the state of the conversation. At termination, results are synthesized together.

Agent orchestrator workflows simplifies this routing pattern through a central orchestrator (`GroupChatManager`) that selectively delegates tasks to the appropriate agents. Each agent monitors the conversation but only contributes when their specific expertise is required. With Phoenix tracing, you get full visibility into the orchestration flow to see which agents engaged, when they were activated, and why.

In this example, we'll build a smart trip planning assistant where subtasks like destination research, hotel booking, and activity suggestions are dynamically sent to the right specialized agent.

By the end of this tutorial, you’ll learn how to:

- Set up multiple specialized AutoGen agents in a `GroupChat`

- Use a `GroupChatManager` to enable dynamic agent routing

- Incorporate human feedback in your agent set up

- Trace and visualize agent interactions using Phoenix

⚠️ You'll need an OpenAI Key for this tutorial.


## Set up Keys and Dependencies


In [ ]:
%pip install -qq pyautogen==0.9 autogen-agentchat~=0.2 openai

In [ ]:
%pip install -qqq arize-phoenix arize-phoenix-otel openinference-instrumentation-openai

In [ ]:
import os
from getpass import getpass

import autogen
import nest_asyncio
import pandas as pd

import phoenix as px

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint")

## Configure Tracing


In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="autogen-agents",
    auto_instrument=True,
)

## Example Orchestrator Task: Travel Agent

This example shows how to build a dynamic travel planning assistant. A `GroupChatManager` coordinates specialized agents to adapt to the user's evolving travel needs.

**User Interaction**:
A `UserProxyAgent` acts as the human user, configured with `human_input_mode="TERMINATE" `and a custom `is_termination_msg` that ends the session when a message ends with TERMINATE.

**Specialized Travel Agents**:
Three AssistantAgents handle specific tasks.

- Flight Planner — suggests flight options.

- Hotel Finder — recommends accommodations.

- Activity Suggester — proposes activities and attractions.

**GroupChat Setup**:
A GroupChat bundles the user and specialized agents, managing message flow with a maximum round limit (ex: 10 rounds).

**Orchestrator**:
The `GroupChatManager` oversees the conversation, routing tasks to the right agent based on context.

![Diagram](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_orchestrator_diagram.png)

## Define Agent

The `llm_config` specifies the configuration used for all the assistant agents.


In [ ]:
llm_config = {
    "model": "gpt-4o",
    "api_key": os.environ["OPENAI_API_KEY"],
}

In [ ]:
# Specialized LLM Agents
flight_planner = autogen.AssistantAgent(
    name="FlightPlanner",
    llm_config=llm_config,
    system_message="You are a flight planning assistant. You help book flights and find the best travel routes. Focus on using freely accessible sources.",
)

hotel_finder = autogen.AssistantAgent(
    name="HotelFinder",
    llm_config=llm_config,
    system_message="You are a hotel booking assistant. You help find the best accommodations. Focus on using freely accessible sources.",
)

activity_suggester = autogen.AssistantAgent(
    name="ActivitySuggester",
    llm_config=llm_config,
    system_message="You are a travel activity expert. You suggest interesting activities and tours in a destination.",
)

In [ ]:
user_proxy = autogen.UserProxyAgent(
    name="UserProxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    system_message="A human user seeking travel planning assistance. Reply TERMINATE when the task is done.",
)

agents = [user_proxy, flight_planner, hotel_finder, activity_suggester]

group_chat = autogen.GroupChat(agents=agents, messages=[], max_round=10)

manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
    system_message="""
    You are a coordinator managing a travel planning discussion between a user, a flight planner, a hotel finder, and an activity suggester.
    Your goal is to ensure the user's request is fully addressed by coordinating the specialists.
    Ensure each specialist contributes relevant information sequentially (e.g., flights first, then hotels, then activities, unless the user specifies otherwise).
    Summarize the final plan. Be sure to reply TERMINATE when the plan is complete.
    """,
)

## Run Agent

In [ ]:
from opentelemetry.trace import StatusCode

tracer = tracer_provider.get_tracer(__name__)
with tracer.start_as_current_span(
    "TravelAgent",
    openinference_span_kind="agent",
) as agent_span:
    agent_span.set_status(StatusCode.OK)
    user_proxy.initiate_chat(
        manager,
        message="I want to plan a 2-day trip to Cabo sometime in October. I'm interested in good food. Find flight options from SFO, suggest mid-range hotels near the city center, and recommend some relevant activities.",
    )

# Let's add some Evaluations (Evals)

In this section we will evaluate Agent Trajectory. 

See https://arize.com/docs/ax/evaluate/agent-trajectory-evaluations 

In [ ]:
df = px.Client().get_spans_dataframe(project_name="autogen-agents", timeout=None)
llm_spans = df[df["span_kind"] == "LLM"]
root_ids = df[df["parent_id"].isna()]["context.trace_id"].unique()
llm_spans.head()

In [ ]:
TRAJECTORY_ACCURACY_PROMPT_WITHOUT_REFERENCE = """
You are a helpful AI bot that checks whether an AI agent's internal trajectory is accurate and effective.

You will be given:
1. The agent's actual trajectory of tool calls
2. You will be given input data from a user that the agent used to make a decision
3. You will be given a tool call definition, what the agent used to make the tool call

An accurate trajectory:
- Progresses logically from step to step
- Follows the golden trajectory where reasonable
- Shows a clear path toward completing a goal
- Is reasonably efficient (doesn't take unnecessary detours)

##

Actual Trajectory:
{tool_calls}

Use Inputs:
{attributes.input.value}

Tool Definitions:
{attributes.llm.tools}

##

Your response must be a single string, either `correct` or `incorrect`, and must not include any additional text.

- Respond with `correct` if the agent's trajectory adheres to the rubric and accomplishes the task effectively.
- Respond with `incorrect` if the trajectory is confusing, misaligned with the goal, inefficient, or does not accomplish the task.
"""

In [ ]:
from typing import Any, Dict


def filter_spans_by_trace_criteria(
    df: pd.DataFrame,
    trace_filters: Dict[str, Dict[str, Any]],
    span_filters: Dict[str, Dict[str, Any]],
) -> pd.DataFrame:
    """Filter spans based on trace-level and span-level criteria.

    Args:
        df: DataFrame with trace data
        trace_filters: Dictionary of column names and filtering criteria for traces
                      Format: {"column_name": {"operator": value}}
                      Supported operators: ">=", "<=", "==", "!=", "contains", "notna", "isna"
        span_filters: Dictionary of column names and filtering criteria for spans
                     Format: {"column_name": {"operator": value}}
                     Same supported operators as trace_filters

    Returns:
        DataFrame with filtered spans from traces that match trace_filters
    """
    all_trace_ids = set(df["context.trace_id"].unique())
    print(f"Total traces: {len(all_trace_ids)}")

    df_copy = df.copy()

    traces_df = df_copy.copy()
    for column, criteria in trace_filters.items():
        if column not in traces_df.columns:
            print(f"Warning: Column '{column}' not found in dataframe")
            continue

        for operator, value in criteria.items():
            if operator == ">=":
                matching_spans = traces_df[traces_df[column] >= value]
            elif operator == "<=":
                matching_spans = traces_df[traces_df[column] <= value]
            elif operator == "==":
                matching_spans = traces_df[traces_df[column] == value]
            elif operator == "!=":
                matching_spans = traces_df[traces_df[column] != value]
            elif operator == "contains":
                matching_spans = traces_df[
                    traces_df[column].str.contains(value, case=False, na=False)
                ]
            elif operator == "isna":
                matching_spans = traces_df[traces_df[column].isna()]
            elif operator == "notna":
                matching_spans = traces_df[traces_df[column].notna()]
            else:
                print(f"Warning: Unsupported operator '{operator}' - skipping")
                continue

            traces_df = matching_spans

    matching_trace_ids = set(traces_df["context.trace_id"].unique())
    print(f"Found {len(matching_trace_ids)} traces matching trace criteria")

    if not matching_trace_ids:
        print("No matching traces found")
        return pd.DataFrame()

    result_df = df[df["context.trace_id"].isin(matching_trace_ids)].copy()

    for column, criteria in span_filters.items():
        if column not in result_df.columns:
            print(f"Warning: Column '{column}' not found in dataframe")
            continue

        for operator, value in criteria.items():
            if operator == ">=":
                result_df = result_df[result_df[column] >= value]
            elif operator == "<=":
                result_df = result_df[result_df[column] <= value]
            elif operator == "==":
                result_df = result_df[result_df[column] == value]
            elif operator == "!=":
                result_df = result_df[result_df[column] != value]
            elif operator == "contains":
                result_df = result_df[result_df[column].str.contains(value, case=False, na=False)]
            elif operator == "isna":
                result_df = result_df[result_df[column].isna()]
            elif operator == "notna":
                result_df = result_df[result_df[column].notna()]
            else:
                print(f"Warning: Unsupported operator '{operator}' - skipping")
                continue

    print(f"Final result: {len(result_df)} spans from {len(matching_trace_ids)} traces")
    return result_df


def extract_tool_calls(output_messages):
    if not output_messages:
        return []

    tool_calls = []
    for message in output_messages:
        if "message.tool_calls" in message:
            for tool_call in message["message.tool_calls"]:
                tool_calls.append({"name": tool_call["tool_call.function.name"]})
    return tool_calls

In [ ]:
from typing import Any, Dict

import pandas as pd

eval_traces = filter_spans_by_trace_criteria(
    df=df,
    trace_filters={"name": {"contains": "agent"}},
    span_filters={"attributes.openinference.span.kind": {"==": "LLM"}},
)

eval_traces.head()

In [ ]:
eval_traces["tool_calls"] = eval_traces["attributes.llm.output_messages"].apply(extract_tool_calls)
eval_traces.head()
full_eval_spans = eval_traces[eval_traces["attributes.llm.tools"].notna()]

In [ ]:
from phoenix.evals import OpenAIModel, llm_classify
from phoenix.trace import suppress_tracing

nest_asyncio.apply()

model = OpenAIModel(
    api_key=os.environ["OPENAI_API_KEY"],
    model="gpt-4o-mini",
    temperature=0.0,
)

rails = ["correct", "incorrect"]

with suppress_tracing():
    eval_results = llm_classify(
        dataframe=full_eval_spans,
        template=TRAJECTORY_ACCURACY_PROMPT_WITHOUT_REFERENCE,
        model=model,
        rails=rails,
        provide_explanation=True,
        verbose=False,
        concurrency=20,
    )

eval_results["score"] = eval_results["label"].apply(lambda x: 1 if x == "correct" else 0)

In [ ]:
import pandas as pd

merged_df = pd.merge(full_eval_spans, eval_results, left_index=True, right_index=True)

merged_df.rename(columns={"context.trace_id": "context.span_id"}, inplace=True)

merged_df.head()

In [ ]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(
        dataframe=merged_df,
        eval_name="Agent Trajectory Accuracy",
    )
)

## View Results in Phoenix

When viewing the traces in Phoenix, you can see how the `GroupChatManager` delegated subtasks to specialized agents step-by-step. The trace shows the order in which each agent responded, making it easy to verify the flow from flight planning to hotel booking to activity suggestions.

![Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_routing_results.png)


Run the cell below to see full tracing results.

In [ ]:
from IPython.display import HTML

HTML("""
<video width="800" height="600" controls>
  <source src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/autogen_router_agent.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")